In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import torch

from gpdm import GPDM
from mocap_labels import WALK_TRIALS, RUN_TRIALS
import select_joints

In [ ]:
print(f"Number of walk trials: {len(WALK_TRIALS)}")
print(f"Number of run trials: {len(RUN_TRIALS)}")

In [ ]:
reduced_columns = WALK_TRIALS[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")


In [ ]:
walk_dfs = [trial.as_dataframe() for trial in WALK_TRIALS]
run_dfs = [trial.as_dataframe() for trial in RUN_TRIALS]

walk_arrs = [df[reduced_columns].to_numpy().astype(np.float32) for df in walk_dfs]
run_arrs = [df[reduced_columns].to_numpy().astype(np.float32) for df in run_dfs]

In [5]:
d = 3 # latent space dimension

# Initial values for hyperparameters
y_lambdas_init = np.ones(DOFs)  # Signal standard deviation for observation GP
y_lengthscales_init = np.ones(d)  # Lengthscales for observation GP
y_sigma_n_init = 1e-2  # Noise standard deviation for observation GP

x_lambdas_init = np.ones(d)  # Signal standard deviation for latent dynamics GP
x_lengthscales_init = np.ones(2*d)  # Lengthscales for latent dynamics GP (since dyn_back_step = 2)
x_sigma_n_init = 1e-2  # Noise standard deviation for latent dynamics GP
x_lin_coeff_init = np.ones(2*d + 1)  # Linear coefficients for latent dynamics GP


In [6]:
# Create GPDM object
walk_gpdm = GPDM(D=DOFs, d=d, 
                  dyn_target='full', dyn_back_step=2, 
                  y_lambdas_init=y_lambdas_init,
                  y_lengthscales_init=y_lengthscales_init, 
                  y_sigma_n_init=y_sigma_n_init, 
                  x_lambdas_init=x_lambdas_init, 
                  x_lengthscales_init=x_lengthscales_init, 
                  x_sigma_n_init=x_sigma_n_init, 
                  x_lin_coeff_init=x_lin_coeff_init, 
                  flg_train_y_lambdas=True, 
                  flg_train_y_lengthscales=True, 
                  flg_train_y_sigma_n=True, 
                  flg_train_x_lambdas=True, 
                  flg_train_x_lengthscales=True, 
                  flg_train_x_sigma_n=True, 
                  flg_train_x_lin_coeff=True, 
                  sigma_n_num_Y=0., 
                  sigma_n_num_X=0., 
                  dtype=torch.float64, 
                  device=torch.device('cpu'))

In [ ]:
# sample 5 random walk sequences for training

walk_arrs_train = random.sample(walk_arrs, 5)

for walk_arr in walk_arrs_train:
    walk_gpdm.add_data(walk_arr)

print('Num. of sequences = '+str(walk_gpdm.num_sequences)+' [Data points = '+str(np.concatenate(walk_gpdm.observations_list, 0).shape[0])+']')


In [ ]:
walk_gpdm.init_X()  
walk_gpdm.train_adam(num_opt_steps=1000, num_print_steps=10, lr=0.01)

walk_gpdm.save('walk_gpdm_config.pth', 'walk_gpdm_state.pth')  

In [ ]:
# Plot the latent space
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
X = walk_gpdm.X.detach().numpy()
ax.plot(X[:, 0], X[:, 1], X[:, 2])